In [14]:
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import re
import time
import unidecode

In [2]:
Base = declarative_base()

In [3]:
class Artist(Base):
    __tablename__ = 'artists'
    id = Column(Integer, primary_key=True)
    name = Column(Text)
    gender = Column(Text)
    birthday = Column(Integer)
    deathday = Column(Integer)
#     hometown = Column(Text)
    location = Column(Text)
    nationality = Column(Text)
    years = relationship('Year', back_populates='artist')
    genes = relationship('Gene', secondary='artists_genes')

In [4]:
class Year(Base):
    __tablename__ = 'years'
    id = Column(Integer, primary_key=True)
    year = Column(Integer)
    rank = Column(Integer)
    artist_id = Column(Integer, ForeignKey('artists.id'))
    artist = relationship('Artist', back_populates='years')
    totalsold = Column(Integer)
    totallots =  Column(Integer)
    maxprice = Column(Integer)

In [5]:
#if no display_name, name
class Gene(Base):
    __tablename__= 'genes'
    id = Column(Integer, primary_key=True)
    name = Column(Text)
    artists = relationship('Artist', secondary='artists_genes')

In [6]:
class ArtistGene(Base):
    __tablename__ = 'artists_genes'
    artist_id = Column(Integer, ForeignKey('artists.id'), primary_key=True)
    gene_id = Column(Integer, ForeignKey('genes.id'), primary_key=True)

In [25]:
engine = create_engine('sqlite:///artsy.db')
Base.metadata.create_all(engine)

In [26]:
Session = sessionmaker(bind=engine)
session = Session()

In [9]:
from artists2010 import clean2010top500

In [ ]:
"-".join(clean2010top500[400]['Name']).lower()

In [10]:
import requests
import json

client_id = '5a2c7ea90a393fa55c65'
client_secret = '5cda60eedbc43c586621c30d6d19210e'

r = requests.post('https://api.artsy.net/api/tokens/xapp_token',
                      data={'client_id': client_id, 'client_secret': client_secret})

token_text = json.loads(r.text)

token = token_text['token']

headers = {'X-Xapp-Token': token}

list_answer = {}
artists = 'qi-baishi'

# api_url='https://api.artsy.net/api/artists/'+artists.lower()
api_url = 'https://api.artsy.net/api/search?offset=0&q=ZHU+DA'
res = requests.get(api_url, headers=headers)
res.encoding = 'utf-8'

In [11]:
res.json()['_embedded']['results'][0]['_links']['self']['href']

'https://api.artsy.net/api/artists/4ee262199de8920001000530'

In [12]:
def verify_artist(json_thing):
    if json_thing.get('type') != 'other_error':
        return True
    elif json_thing.get('type') == 'other_error':
        return False

# Order is fine, gotta figure 404s
def query_json(artist_name):
    artist_name = [re.sub("[')(.]", '', x)  for x in artist_name]
    artist_name = [unidecode.unidecode(y) for y in artist_name]
    name = "-".join(artist_name)
    api_url='https://api.artsy.net/api/artists/'+name.lower()
    res = requests.get(api_url, headers=headers)
    res.encoding = 'utf-8'
    verifiable = verify_artist(res.json())
    if  verifiable == True:
        #debugging
        print('verified true')
        print(res)
        #########
        return res.json()
    elif verifiable == False :
        new_name = list(reversed(artist_name))
        name = "-".join(new_name)
        api_url='https://api.artsy.net/api/artists/'+name.lower()
        res = requests.get(api_url, headers=headers)
        res.encoding = 'utf-8'
        #debugging
        print('flipped name')
        print(res)
        #########
        if res.status_code == 404:
            search_url = 'https://api.artsy.net/api/search?q='+ name
            res = requests.get(search_url, headers=headers)
            res.encoding = 'utf-8'
            t = res.json()['_embedded']['results'][0]['type']
            if t  != 'artist':
                return {'name' : " ".join(artist_name), 'skip':True}  
            api_url = res.json()['_embedded']['results'][0]['_links']['self']['href']
            res = requests.get(api_url, headers=headers)
            res.encoding = 'utf-8'
            #debugging
            print('search query')
            print(res)
            ##########
            return res.json()
        return res.json()
    

In [15]:
query_json(['Kasimir', 'Sevrinovitch', 'Malevich'])

flipped name
<Response [404]>
search query
<Response [200]>


{'id': '4d8b92694eb68a1b2c0000ac',
 'slug': 'kasimir-severinovich-malevich',
 'created_at': '2010-09-09T17:10:23+00:00',
 'updated_at': '2018-11-13T13:37:10+00:00',
 'name': 'Kasimir Severinovich Malevich',
 'sortable_name': 'Malevich Kasimir Severinovich',
 'gender': 'male',
 'biography': None,
 'birthday': '1879',
 'deathday': '1935',
 'hometown': 'Kiev, Ukraine',
 'location': 'Moscow, Russia',
 'nationality': 'Russian',
 'image_versions': ['four_thirds', 'large', 'square', 'tall'],
 '_links': {'thumbnail': {'href': 'https://d32dm0rphc51dk.cloudfront.net/Pwq2eoTMbl_vINVj2ooM3A/four_thirds.jpg'},
  'image': {'href': 'https://d32dm0rphc51dk.cloudfront.net/Pwq2eoTMbl_vINVj2ooM3A/{image_version}.jpg',
   'templated': True},
  'self': {'href': 'https://api.artsy.net/api/artists/4d8b92694eb68a1b2c0000ac'},
  'permalink': {'href': 'http://www.artsy.net/artist/kasimir-severinovich-malevich'},
  'artworks': {'href': 'https://api.artsy.net/api/artworks?artist_id=4d8b92694eb68a1b2c0000ac'},
  '

In [16]:
def create_genes(artist_json):
    genes_toadd = []
    if artist_json.get('skip'):
        return []
    else:
        x = artist_json['_links']['genes']['href']
    genes = requests.get(x, headers=headers)
    gene_json = genes.json()['_embedded']['genes']
    artist_genes = []
    for gene in gene_json:
        if gene['display_name'] != None:
            artist_genes.append(gene['display_name']) 
        else: 
            artist_genes.append(gene['name'])
    gene_objects = []
    for gene in artist_genes:
        existing = session.query(Gene).filter(Gene.name == gene).first()
        if existing:
            gene_objects.append(existing)
        else:
            gene_objects.append(Gene(name=gene))
    return gene_objects

In [27]:
session.query(Artist).filter(Artist.name == None).first()

In [18]:
def create_artist(artist_name):
    artist_json = query_json(artist_name)
    existing = session.query(Artist).filter(Artist.name == artist_json['name']).first()
    if existing:
        return existing
    else:
        n = artist_json.get('name')
        g = artist_json.get('gender')
        bday = artist_json.get('birthday')
        dday = artist_json.get('deathday')
        loc = artist_json.get('location')
        nat = artist_json.get('nationality')
        gen = create_genes(artist_json)
        return Artist(name=n, gender=g, birthday=bday,
                     deathday=dday, location=loc, nationality=nat, genes=gen)

In [19]:
def create_database_year(year):
    for row in year:
        print(row['Rank'])
        name = row['Name']
        artist_object = create_artist(name)
        time.sleep(0.75)
        year = Year(year=row['SaleYear'],
                    rank=row['Rank'],
                    artist=artist_object,
                    totalsold=row['TotalSold'],
                    totallots=row['TotalLots'],
                    maxprice=row['MaxPrice'])
        session.add(year)
        session.commit()

In [20]:
from artists2017 import clean2017top500

In [28]:
create_database_year(clean2017top500[190:200])

191
verified true
<Response [200]>
192
verified true
<Response [200]>
193
verified true
<Response [200]>
194
flipped name
<Response [404]>
search query
<Response [200]>
195
verified true
<Response [200]>
196
flipped name
<Response [404]>
search query
<Response [200]>
197
verified true
<Response [200]>
198
verified true
<Response [200]>
199
verified true
<Response [200]>
200
verified true
<Response [200]>
